# Quantum SVM with Quantum Circuits

In this tutorial, a quantum version of the Support Vector Machine (SVM) algorithm based on a Quantum Circuit (QC) is presented.

## Initial Steps

### Library Installation

In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
  !pip install qiskit

### Account Configuration

In [ ]:
import qiskit
token = 'TOKEN'
qiskit.IBMQ.save_account(token)
account = qiskit.IBMQ.load_account()
provider = qiskit.IBMQ.get_provider(hub='ibm-q')
backend = provider.get_backend('ibmq_qasm_simulator')
print(provider, backend)

### Library Imports

In [ ]:
import numpy as np
import glob
import matplotlib.pyplot as plt
from matplotlib import colors
from functools import reduce
from sklearn import preprocessing
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from qiskit import BasicAer, IBMQ
from qiskit.circuit.library import ZZFeatureMap, PauliFeatureMap
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import QSVM
from qiskit.aqua.utils import split_dataset_to_data_and_labels, map_label_to_class_name

## Quantum SVM Algorithm

The presented quantum implementation of the SVM algorithm based on quantum circuits is already available in Qiskit (QSVM). The reference paper is _V. Havlicek et al., “Supervised Learning with Quantum-Enhanced Feature Spaces_.

A quantum feature map $\mathcal{U}_{\Phi(\mathbf{x})}$ is constructed, i.e., a variational circuit that encodes a vector $\mathbf{x}$ into the quantum state $|\Phi(\mathbf{x})\rangle$. The quantum feature map is applied to an initial state $|{0}^D\rangle$, with $D$ number of qubits, equal to the dimension of the samples. 

### Dataset and Parameter Definition

A simple 2-dimensional dataset is considered.

In [ ]:
X_train = np.array([
       [ 0.0401,  0.1415],
       [-0.1211, -0.0228],
       [-0.1688,  0.2911],
       [ 0.2293,  0.7856],
       [-1.0831, -0.1049],
       [ 0.1626, -1.0458],
       [ 0.311 , -0.1032],
       [-0.9376, -0.2677],
       [ 0.7176, -0.7998],
       [-0.0197,  0.3323],
       [-0.1306,  0.1053],
       [ 0.0811, -0.1263],
       [ 0.3391,  0.1625],
       [ 0.6602,  0.0843],
       [-0.1588,  0.1835],
       [-0.3887,  0.2282],
       [ 0.2555,  0.0709],
       [ 0.1717, -0.9481],
       [-0.0288, -0.1503],
       [-0.7354,  0.5788]])
Y_train = np.array(
      [ 1.,  1.,  1., -1., -1., -1.,  1., -1., -1.,  1.,  1.,  1.,  1.,
       -1.,  1.,  1.,  1., -1.,  1., -1.])
X_test = np.array([
       [-0.6483, -1.2539],
       [ 0.1448,  1.0718],
       [ 0.803 , -0.7706],
       [-0.1384, -0.9474],
       [-0.0025,  0.0779],
       [-0.686 ,  0.9516],
       [ 0.1013, -0.5189],
       [ 0.8678,  0.0948],
       [-0.0021, -0.082 ],
       [-1.0192, -0.3854],
       [ 0.1661, -0.1268],
       [ 0.2193,  0.0973],
       [-0.1468,  1.0086],
       [-0.0195,  0.503 ],
       [ 0.0475,  0.3345],
       [ 0.7212, -0.1479],
       [-0.5998,  0.5036],
       [-0.1889, -0.0499],
       [ 0.9087,  0.525 ],
       [ 0.5112, -0.4113]]
)
Y_test = np.array(
      [-1., -1., -1., -1.,  1., -1., 1., -1.,  1., -1.,  1.,  1., -1.,  1.,  1.,
       -1., -1.,  1., -1., -1.]
)

plt.figure(0)
plt.title('Training set')
plt.xlim(-2, 2)
plt.ylim(-2, 2)
plot = plt.scatter(X_train[:, 0], X_train[:, 1], c=Y_train, s=50, cmap='autumn')

In [34]:
feature_dim = np.size(X_train[0])

Y_train_01 = np.where(Y_train==-1, 0, Y_train)
Y_test_01 = np.where(Y_test==-1, 0, Y_test)
X_train_0_indices = [i for i, y in enumerate(Y_train_01) if y == 0]
X_train_1_indices = [i for i, y in enumerate(Y_train_01) if y == 1]
X_test_0_indices = [i for i, y in enumerate(Y_test_01) if y == 0]
X_test_1_indices = [i for i, y in enumerate(Y_test_01) if y == 1]
training_input = {'0': X_train[X_train_0_indices], '1': X_train[X_train_1_indices]}
test_input = {'0': X_test[X_test_0_indices], '1': X_test[X_test_1_indices]}
datapoints = [X_test,Y_test_01]

### Circuit Definition

In [ ]:
def linear_feature_map(x: np.ndarray) -> float:
    coeff = x[0] if len(x) == 1 else reduce(lambda m, n: m*n, x)
    return coeff
feature_map = ZZFeatureMap(feature_dimension=feature_dim, reps=2, data_map_func=linear_feature_map, entanglement='linear')
print(feature_map)

### Problem Submission

In [29]:
seed = 1000
qsvm = QSVM(feature_map, training_input, test_input, datapoints[0])
backend = provider.get_backend('ibmq_qasm_simulator')
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=seed, seed_transpiler=seed)
result = qsvm.run(quantum_instance)

In [ ]:
print(f'Testing success ratio: {result["testing_accuracy"]}')
print('Prediction from datapoints set:')
print(f'  True classes:        {map_label_to_class_name(datapoints[1], qsvm.label_to_class)}')
print(f'  Predicted classes:   {result["predicted_classes"]}')
predicted_labels = result["predicted_labels"]
print(f'  success rate: {100*np.count_nonzero(predicted_labels == datapoints[1])/len(predicted_labels)}%')
print(f'  F1 score: {f1_score(datapoints[1], result["predicted_labels"])}')
print(f'  Accuracy: {accuracy_score(datapoints[1], result["predicted_labels"])}')